# 📚 Updated Audiobook Creator Notebook

Welcome! This notebook has been updated with Tier 1 UI/UX enhancements to create a more intuitive and robust experience.

---
### Key Improvements:
**1. Accordion Layout:** The UI is now organized into collapsible sections for a cleaner, step-by-step workflow.

**2. State Management:** Configuration widgets are automatically disabled during audiobook generation to prevent mid-process changes and provide clear visual feedback.

**3. Dynamic Option Visibility:** The "Add Emotion Tags" option now correctly appears only when the `orpheus` TTS model is selected.

---
### Instructions:
1.  Run the cells in order.
2.  The final cell will display the interactive UI.
3.  Work through the accordion steps to generate your audiobook.

## 1. Initial Setup & Package Installation

This cell ensures all required Python packages are installed in your environment. It will only install packages that are not already present.

In [ ]:
# Install required packages if missing
import sys
import subprocess

def install_if_missing(package):
    try:
        __import__(package)
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])

# List of required packages
required_packages = ["openai", "tqdm", "pydub", "word2number", "python-dotenv", "nest_asyncio", "ipywidgets"]
for pkg in required_packages:
    install_if_missing(pkg)

print("All required packages are installed.")

## 2. Core Imports

This cell imports the necessary libraries for the application to run. It also applies a patch to allow `asyncio` to work correctly within the Jupyter environment.

In [ ]:
import nest_asyncio
import ipywidgets as widgets
from IPython.display import display, Audio, FileLink, clear_output
import os
from dotenv import load_dotenv
import asyncio

# This is a placeholder for your actual generation logic.
# Make sure you have a 'generate_audiobook.py' file with this function.
try:
    from generate_audiobook import process_audiobook_generation
except ImportError:
    print("Warning: 'generate_audiobook.py' not found. Creating a placeholder function.")
    async def process_audiobook_generation(*args, **kwargs):
        yield "Starting placeholder generation..."
        for i in range(101):
            await asyncio.sleep(0.05)
            yield f"Processing... Progress: {i}%"
        # Create a dummy file for download testing
        os.makedirs('generated_audiobooks', exist_ok=True)
        dummy_filepath = 'generated_audiobooks/audiobook.placeholder.mp3'
        with open(dummy_filepath, 'w') as f:
            f.write("This is a placeholder file.")
        yield f"Placeholder file created successfully at {dummy_filepath}"

# Apply nest_asyncio to allow nested event loops in Jupyter environments
nest_asyncio.apply()

# Load environment variables from .env file
load_dotenv()

## 3. Widget Definitions & UI Layout

Here, we define all the interactive UI components (widgets) and organize them into a clean, collapsible `Accordion` layout.

In [ ]:
# --- Define All Widgets ---

# Step 1: Book Selection
book_dir = './sample_book_and_audio'
os.makedirs(book_dir, exist_ok=True) # Ensure directory exists
book_files = [f for f in os.listdir(book_dir) if f.endswith(('.epub', '.pdf', '.txt'))]
book_dropdown = widgets.Dropdown(options=book_files, description='Select Book:', value=book_files[0] if book_files else None, style={'description_width': 'initial'})
upload_widget = widgets.FileUpload(accept='.epub,.pdf,.txt', multiple=False, description='Or Upload New')

# Step 2: Audiobook Options
tts_model = widgets.Dropdown(options=['kokoro', 'orpheus'], description='TTS Model:', value='kokoro', style={'description_width': 'initial'})
voice_option = widgets.ToggleButtons(options=['Single Voice', 'Multi-Voice'], description='Voice Mode:')
output_format = widgets.Dropdown(options=['aac', 'm4a', 'mp3', 'wav', 'opus', 'flac', 'pcm', 'M4B (Chapters & Cover)'], description='Format:', style={'description_width': 'initial'})
narrator_gender = widgets.ToggleButtons(options=['male', 'female'], description='Narrator:')
add_emotion_tags = widgets.Checkbox(value=False, description='Add Emotion Tags (Orpheus only)', indent=False)
advanced_toggle = widgets.Checkbox(value=False, description='Show Advanced Options', indent=False)

# Step 3: Generation Controls
generate_button = widgets.Button(description='Generate Audiobook', button_style='success', icon='cogs')
cancel_button = widgets.Button(description='Cancel Generation', button_style='danger', icon='stop', disabled=True)
reset_button = widgets.Button(description='Reset Options', button_style='warning', icon='refresh')
progress_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info', orientation='horizontal')
notification = widgets.HTML(value="", placeholder='Notifications will appear here.')
progress_output = widgets.Output(layout={'border': '1px solid gray', 'height': '300px', 'overflow_y': 'auto', 'margin_top': '10px'})

# Step 4: Review & Download
audio_select = widgets.Dropdown(description='Audiobook:', style={'description_width': 'initial'})
play_button = widgets.Button(description='Play', button_style='info', icon='play')
download_button = widgets.Button(description='Download', button_style='primary', icon='download')
audio_output = widgets.Output()

# --- Group Widgets into Accordion Panes ---

# Tier 1, Suggestion 1: Implement an Accordion Layout
step1_content = widgets.VBox([
    widgets.Label("Select a pre-existing book or upload a new one (.epub, .pdf, .txt)."),
    widgets.HBox([book_dropdown, upload_widget])
])

step2_content = widgets.VBox([
    tts_model,
    voice_option,
    output_format,
    narrator_gender,
    add_emotion_tags,
    advanced_toggle
])

step3_content = widgets.VBox([
    widgets.HBox([generate_button, cancel_button, reset_button]),
    progress_bar,
    notification,
    progress_output
])

step4_content = widgets.VBox([
    widgets.HBox([audio_select, play_button, download_button]),
    audio_output
])

accordion = widgets.Accordion(children=[step1_content, step2_content, step3_content, step4_content])
accordion.set_title(0, '1. Select or Upload Book')
accordion.set_title(1, '2. Audiobook Options')
accordion.set_title(2, '3. Generate')
accordion.set_title(3, '4. Review & Download')
accordion.selected_index = 0 # Start with the first section open

# --- List of widgets to manage state ---
# Tier 1, Suggestion 2: Improve Visual Feedback and State Management
config_widgets = [
    book_dropdown, upload_widget, tts_model, voice_option,
    output_format, narrator_gender, add_emotion_tags, advanced_toggle
]

## 4. Widget Logic and Event Handlers

This section contains all the functions that respond to user interactions, such as button clicks and dropdown changes. This is the 'brain' of the application.

In [ ]:
# --- Global State ---
is_generating = False

# --- Handler for Upload ---
def handle_upload(change):
    """Saves uploaded file and updates the book dropdown."""
    for filename, fileinfo in upload_widget.value.items():
        filepath = os.path.join(book_dir, filename)
        with open(filepath, 'wb') as f:
            f.write(fileinfo['content'])
        
        # Update dropdown options
        current_options = list(book_dropdown.options)
        if filename not in current_options:
            current_options.append(filename)
            book_dropdown.options = current_options
        book_dropdown.value = filename
        upload_widget.value.clear() # Clear the upload widget after success
upload_widget.observe(handle_upload, names='value')


# --- Handler for Dynamic Option Visibility ---
# Tier 1, Suggestion 3: Dynamic Option Visibility
def on_tts_model_change(change):
    """Shows or hides the emotion tags checkbox based on the selected TTS model."""
    if change.get('new') == 'orpheus':
        add_emotion_tags.layout.display = 'flex'
    else:
        add_emotion_tags.layout.display = 'none'
        add_emotion_tags.value = False  # Also reset the value when it's hidden
tts_model.observe(on_tts_model_change, names='value')


# --- Handler for State Management ---
# Tier 1, Suggestion 2 (Helper Function)
def set_ui_for_generation(is_active: bool):
    """Disables or enables UI controls based on generation state."""
    global is_generating
    is_generating = is_active
    for widget in config_widgets:
        widget.disabled = is_active
    generate_button.disabled = is_active
    reset_button.disabled = is_active
    cancel_button.disabled = not is_active


# --- Handlers for Generation Buttons ---
def on_generate_clicked(b):
    """Starts the audiobook generation process."""
    if not book_dropdown.value:
        notification.value = "<b style='color:red;'>Error: Please select or upload a book first.</b>"
        return

    set_ui_for_generation(True)
    accordion.selected_index = 2 # Switch view to the 'Generate' tab
    
    # Clear previous run outputs
    progress_output.clear_output()
    progress_bar.value = 0
    notification.value = ""
    
    # Get configuration
    book_path = os.path.join(book_dir, book_dropdown.value)
    tts_env = tts_model.value

    with progress_output:
        print(f"Starting generation for: {book_path} (TTS Model: {tts_env})")
        
        async def run_generation():
            try:
                async for update in process_audiobook_generation(
                    voice_option.value,
                    narrator_gender.value,
                    output_format.value,
                    book_path,
                    add_emotion_tags.value
                ):
                    if not is_generating:
                        print("Generation cancelled by user.")
                        notification.value = "<b style='color:orange;'>Generation cancelled.</b>"
                        break
                    
                    print(update) # Log to progress output area
                    
                    if "Progress:" in update:
                        try:
                            percent = float(update.split("Progress:")[1].split("%")[0])
                            progress_bar.value = int(percent)
                        except (ValueError, IndexError):
                            pass
                    
                    if "Error:" in update:
                        notification.value = f"<b style='color:red;'>{update}</b>"
                    
                    if "successfully" in update or "created" in update:
                        notification.value = f"<b style='color:green;'>{update}</b>"
                        update_audio_file_list() # Refresh download list
                        accordion.selected_index = 3 # Switch to download tab on success
            
            except Exception as e:
                print(f"An unhandled error occurred: {e}")
                notification.value = f"<b style='color:red;'>Error: {e}</b>"
        
        try:
            # Run the async task
            asyncio.get_event_loop().run_until_complete(run_generation())
        finally:
            # Always re-enable UI controls when finished or on error
            set_ui_for_generation(False)

generate_button.on_click(on_generate_clicked)


def on_cancel_clicked(b):
    """Sets the flag to stop the generation process."""
    global is_generating
    if is_generating:
        is_generating = False # The running loop will detect this and stop
        notification.value = "<b style='color:orange;'>Cancellation signal sent. Finishing current task...</b>"
cancel_button.on_click(on_cancel_clicked)


def on_reset_clicked(b):
    """Resets all options to their default values."""
    book_dropdown.value = book_files[0] if book_files else None
    tts_model.value = 'kokoro'
    voice_option.value = 'Single Voice'
    output_format.value = 'aac'
    narrator_gender.value = 'male'
    add_emotion_tags.value = False
    advanced_toggle.value = False
    progress_output.clear_output()
    progress_bar.value = 0
    notification.value = ""
reset_button.on_click(on_reset_clicked)


# --- Handlers for Review & Download ---
def update_audio_file_list():
    """Scans for generated audiobooks and updates the dropdown."""
    audio_dir = 'generated_audiobooks'
    os.makedirs(audio_dir, exist_ok=True)
    try:
        audio_files = sorted(
            [os.path.join(audio_dir, f) for f in os.listdir(audio_dir)], 
            key=os.path.getmtime, 
            reverse=True
        )
        audio_select.options = audio_files
        audio_select.value = audio_files[0] if audio_files else None
    except Exception as e:
        audio_output.clear_output()
        with audio_output:
            print(f"Error updating file list: {e}")

def on_play_clicked(b):
    """Plays the selected audiobook file."""
    audio_output.clear_output()
    if audio_select.value:
        with audio_output:
            print(f"Playing: {audio_select.value}")
            display(Audio(filename=audio_select.value))
play_button.on_click(on_play_clicked)


def on_download_clicked(b):
    """Provides a download link for the selected audiobook."""
    audio_output.clear_output()
    if audio_select.value:
        with audio_output:
            display(FileLink(audio_select.value, result_html_prefix="<b>Download Link: </b>"))
download_button.on_click(on_download_clicked)

## 5. Display the User Interface

Run this final cell to set the initial state of the UI and render the interactive Audiobook Creator.

In [ ]:
# --- Set Initial States ---

# Trigger the handler once to set the initial visibility of the emotion tags checkbox
on_tts_model_change({'new': tts_model.value})

# Populate the list of downloadable files on startup
update_audio_file_list()

# --- Display the main UI ---
print("Audiobook Creator UI is ready.")
display(accordion)